In [80]:
#оставить
!pip install numpy==1.26.4 pandas==2.2.2 pandas_ta ta

In [81]:
#оставить
import pandas as pd
df=pd.read_csv("EURUSD_Candlestick_5_M_ASK_30.09.2019-30.09.2022.csv")
df

,Gmt time,Open,High,Low,Close,Volume
0,30.09.2019 00:00:00.000,1.09425,1.09426,1.09405,1.09406,585.10
1,30.09.2019 00:05:00.000,1.09408,1.09414,1.09401,1.09409,289.39
2,30.09.2019 00:10:00.000,1.09410,1.09423,1.09408,1.09410,276.24
3,30.09.2019 00:15:00.000,1.09409,1.09410,1.09388,1.09389,439.29
4,30.09.2019 00:20:00.000,1.09390,1.09395,1.09388,1.09395,341.23
...,...,...,...,...,...,...
225081,30.09.2022 20:35:00.000,0.98028,0.98034,0.98001,0.98022,624.12
225082,30.09.2022 20:40:00.000,0.98023,0.98047,0.98007,0.98030,408.20
225083,30.09.2022 20:45:00.000,0.98026,0.98034,0.98019,0.98031,317.29
225084,30.09.2022 20:50:00.000,0.98031,0.98067,0.97987,0.97999,1472.13


In [82]:
#оставить
df["Gmt time"]=df["Gmt time"].str.replace(".000","")
df['Gmt time']=pd.to_datetime(df['Gmt time'],format='%d.%m.%Y %H:%M:%S')
df.set_index("Gmt time", inplace=True)
df=df[df.High!=df.Low]
len(df)

224989

In [83]:
#оставить
import pandas_ta as ta

# Создаем явную копию DataFrame, чтобы избежать SettingWithCopyWarning
df = df.copy()

# Удаляем существующие столбцы Bollinger Bands, если они есть
bb_cols_to_drop = ['BBL_14_2.0', 'BBM_14_2.0', 'BBU_14_2.0', 'BBB_14_2.0', 'BBP_14_2.0']
existing_bb_cols = [col for col in bb_cols_to_drop if col in df.columns]
if existing_bb_cols:
    df = df.drop(columns=existing_bb_cols)

df["VWAP"]=ta.vwap(df.High, df.Low, df.Close, df.Volume)
df['RSI']=ta.rsi(df.Close, length=16)
my_bbands = ta.bbands(df.Close, length=14, std=2.0)
df=df.join(my_bbands)

In [94]:
#оставить
import numpy as np

# Используем параметры backcandles_buy и backcandles_sell из ячейки параметров
# backcandles определен в ячейке параметров

df['Max_Open_Close'] = np.maximum(df['Open'], df['Close'])
df['Min_Open_Close'] = np.minimum(df['Open'], df['Close'])
df['upt'] = 1
df['dnt'] = 1
df.loc[(df['Max_Open_Close'] >= df['VWAP']), 'dnt'] = 0
df.loc[(df['Min_Open_Close'] <= df['VWAP']), 'upt'] = 0

# Используем backcandles_buy для расчета sig_upt
df['sig_upt'] = df['upt'].rolling(backcandles_buy + 1, min_periods=1).min()
# Используем backcandles_sell для расчета sig_dnt
df['sig_dnt'] = df['dnt'].rolling(backcandles_sell + 1, min_periods=1).min()

df['VWAPSignal'] = 0
df.loc[(df['sig_upt'] == 1) & (df['sig_dnt'] == 1), 'VWAPSignal'] = 3
df.loc[(df['sig_upt'] == 1) & (df['sig_dnt'] == 0), 'VWAPSignal'] = 2
df.loc[(df['sig_upt'] == 0) & (df['sig_dnt'] == 1), 'VWAPSignal'] = 1

# The following columns were temporary and can be dropped
df = df.drop(columns=['Max_Open_Close', 'Min_Open_Close', 'upt', 'dnt', 'sig_dnt', 'sig_upt'])

In [95]:
#оставить
# Выводим столбцы df перед попыткой доступа к TotalSignal
print("Столбцы DataFrame перед выполнением ячейки 65435b94:")
print(df.columns)

df[df.TotalSignal!=0].count()

Столбцы DataFrame перед выполнением ячейки 65435b94:
Index(['Open', 'High', 'Low', 'Close', 'Volume', 'VWAP', 'RSI', 'BBL_14_2.0',
       'BBM_14_2.0', 'BBU_14_2.0', 'BBB_14_2.0', 'BBP_14_2.0', 'VWAPSignal',
       'TotalSignal', 'pointposbreak'],
      dtype='object')


,0
Open,4374
High,4374
Low,4374
Close,4374
Volume,4374
VWAP,4374
RSI,4374
BBL_14_2.0,4374
BBM_14_2.0,4374
BBU_14_2.0,4374


In [96]:
#оставить
import numpy as np
def pointposbreak(x):
    if x['TotalSignal']==1:
        return x['High']+1e-4
    elif x['TotalSignal']==2:
        return x['Low']-1e-4
    else:
        return np.nan

df['pointposbreak'] = df.apply(lambda row: pointposbreak(row), axis=1)

In [97]:
#оставить
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
st=10400
dfpl = df[st:st+350]
dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.VWAP,
                           line=dict(color='blue', width=1),
                           name="VWAP"),
                go.Scatter(x=dfpl.index, y=dfpl['BBL_14_2.0'],
                           line=dict(color='green', width=1),
                           name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_14_2.0'],
                           line=dict(color='green', width=1),
                           name="BBU")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=10, color="MediumPurple"),
                name="Signal")
fig.show()

In [98]:
#оставить
# Используем весь DataFrame df для расчета индикаторов, включая ATR
# dfpl = df[:75000].copy() # Эту строку можно удалить или закомментировать, если хотим использовать весь df

import pandas_ta as ta

# Убедимся, что dfpl определен и является копией, чтобы избежать SettingWithCopyWarning
# Если вы хотите использовать только часть данных для бэктеста, раскомментируйте следующую строку:
dfpl = df.copy() # Используем копию всего df для начала
# dfpl = df[:75000].copy() # Или используйте срез, если нужно

# Рассчитываем ATR на DataFrame dfpl
dfpl['ATR']=ta.atr(dfpl.High, dfpl.Low, dfpl.Close, length=7)

# help(ta.atr)
def SIGNAL():
    # Возвращаем столбец TotalSignal из DataFrame dfpl
    return dfpl.TotalSignal

# Проверим, что столбец ATR добавлен
# display(dfpl.head())

In [60]:
#оставить
!pip install backtesting

In [106]:
#оставить
# @title Параметры торговой стратегии
backcandles_buy = 7 # @param {type:"integer"}
backcandles_sell = 5 # @param {type:"integer"}
bb_length = 14 # @param {type:"integer"}
bb_std = 2.0 # @param {type:"number"}
rsi_length = 16 # @param {type:"integer"}
rsi_buy_level = 55 # @param {type:"integer"}
rsi_sell_level = 45 # @param {type:"integer"}
initial_cash = 10000.0 # @param {type:"number"}
position_size_percentage = 99.0 # @param {type:"number"}
margin = 0.1 # @param {type:"number"}
TPSLRatio = 2.0 # @param {type:"number"}

In [107]:
# @title
#оставить
import re

# Удаляем существующие столбцы Bollinger Bands, если они есть.
# Используем регулярное выражение для поиска столбцов, начинающихся с BBL_, BBM_, BBU_, BBB_, BBP_
# с последующим подчеркиванием и числовыми значениями, чтобы удалить любые предыдущие BB колонки
# независимо от их параметров length и std.
bb_col_pattern = re.compile(r'BB[LUMBP]_\d+_\d+\.?\d*')
existing_bb_cols = [col for col in df.columns if bb_col_pattern.match(col)]

if existing_bb_cols:
    df = df.drop(columns=existing_bb_cols)

# Расчет VWAP. Этот расчет не зависит от параметров BB или RSI.
# Создаем явную копию DataFrame перед добавлением новых столбцов, чтобы избежать SettingWithCopyWarning.
# Убедимся, что мы работаем с актуальным DataFrame после возможного удаления BB колонок.
df = df.copy()
df["VWAP"] = ta.vwap(df.High, df.Low, df.Close, df.Volume)

# Расчет RSI, используя параметр rsi_length из ячейки параметров.
df['RSI'] = ta.rsi(df.Close, length=rsi_length)

# Расчет Bollinger Bands, используя параметры bb_length и bb_std из ячейки параметров.
my_bbands = ta.bbands(df.Close, length=bb_length, std=bb_std)

# Присоединяем столбцы Bollinger Bands к основному DataFrame.
# Теперь, когда старые BB колонки удалены, присоединение должно пройти без конфликтов.
df = df.join(my_bbands)

# Отобразим первые несколько строк, чтобы проверить добавление новых столбцов индикаторов.
display(df.head())

,Open,High,Low,Close,Volume,VWAP,RSI,VWAPSignal,TotalSignal,pointposbreak,BBL_14_2.0,BBM_14_2.0,BBU_14_2.0,BBB_14_2.0,BBP_14_2.0
Gmt time,,,,,,,,,,,,,,,
2019-09-30 00:00:00,1.09425,1.09426,1.09405,1.09406,585.10,1.094123,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-30 00:05:00,1.09408,1.09414,1.09401,1.09409,289.39,1.094109,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-30 00:10:00,1.09410,1.09423,1.09408,1.09410,276.24,1.094116,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-30 00:15:00,1.09409,1.09410,1.09388,1.09389,439.29,1.094072,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-30 00:20:00,1.09390,1.09395,1.09388,1.09395,341.23,1.094046,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
# @title
#оставить
# Обновляем функцию TotalSignal для использования rsi_buy_level и rsi_sell_level
def TotalSignal(l):
    # Проверяем условие для сигнала на покупку (2):
    # VWAPSignal должен быть равен 2 (цена выше VWAP в течение backcandles),
    # цена закрытия должна быть ниже или равна нижней полосе Боллинджера,
    # и RSI должен быть ниже или равен rsi_buy_level из параметров.
    # Исправляем название колонки BB, чтобы включить десятичную точку в std
    if (df.VWAPSignal.iloc[l] == 2
        and df.Close.iloc[l] <= df[f'BBL_{bb_length}_{bb_std}'].iloc[l]
        and df.RSI.iloc[l] <= rsi_buy_level):
            return 2
    # Проверяем условие для сигнала на продажу (1):
    # VWAPSignal должен быть равен 1 (цена ниже VWAP в течение backcandles),
    # цена закрытия должна быть выше или равна верхней полосе Боллинджера,
    # и RSI должен быть выше или равен rsi_sell_level из параметров.
    # Исправляем название колонки BB, чтобы включить десятичную точку в std
    if (df.VWAPSignal.iloc[l] == 1
        and df.Close.iloc[l] >= df[f'BBU_{bb_length}_{bb_std}'].iloc[l]
        and df.RSI.iloc[l] >= rsi_sell_level):
            return 1
    # Если ни одно из условий не выполнено, нет торгового сигнала (0).
    return 0

# Пересчитываем TotalSignal для всего DataFrame, используя обновленную функцию.
# Начинаем с start_index, учитывая зависимости индикаторов.
TotSignal = [0]*len(df)
start_index = max(backcandles, bb_length, rsi_length) # Учитываем зависимости индикаторов

for row in range(start_index, len(df)):
    TotSignal[row] = TotalSignal(row)

df['TotalSignal'] = TotSignal

# Отобразим количество сигналов, чтобы проверить, что они были сгенерированы с новыми параметрами.
display(df[df.TotalSignal!=0].count())

# Выводим столбцы df после добавления TotalSignal
print("\nСтолбцы DataFrame после выполнения ячейки rL84xCmYBTfA:")
print(df.columns)

,0
Open,4419
High,4419
Low,4419
Close,4419
Volume,4419
VWAP,4419
RSI,4419
VWAPSignal,4419
TotalSignal,4419
pointposbreak,4374



Столбцы DataFrame после выполнения ячейки rL84xCmYBTfA:
Index(['Open', 'High', 'Low', 'Close', 'Volume', 'VWAP', 'RSI', 'VWAPSignal',
       'TotalSignal', 'pointposbreak', 'BBL_14_2.0', 'BBM_14_2.0',
       'BBU_14_2.0', 'BBB_14_2.0', 'BBP_14_2.0'],
      dtype='object')


In [109]:
# @title
#оставить
from backtesting import Strategy
from backtesting import Backtest
import pandas as pd
import numpy as np

class MyStrat(Strategy):
    # Параметры стратегии. Backtest может автоматически привязать глобальные
    # переменные с теми же именами к этим атрибутам.
    rsi_buy_level = 45
    rsi_sell_level = 55
    TPSLRatio = 2.0
    initial_cash = 10000.0 # Этот параметр используется в Backtest, не в стратегии напрямую
    position_size_percentage = 99.0 # Этот параметр используется в Backtest, не в стратегии напрямую
    margin = 0.1 # Этот параметр используется в Backtest, не в стратегии напрямую


    def init(self):
        super().init()
        # Получаем сигнал TotalSignal.
        # Этот сигнал уже учитывает backcandles_buy и backcandles_sell,
        # а также bb_length, bb_std, rsi_length, rsi_buy_level и rsi_sell_level
        # из глобальных переменных, используемых при его расчете вне стратегии.
        self.signal1 = self.I(SIGNAL)


    def next(self):
        super().next()
        # Размер стоп-лосса на основе ATR.
        # Убедимся, что ATR доступен и не NaN для текущей свечи.
        if len(self.data.ATR) == 0 or np.isnan(self.data.ATR[-1]):
            return # Пропускаем, если ATR недоступен

        slatr = 1.2 * self.data.ATR[-1]

        # Получаем TPSLRatio из атрибута класса.
        TPSLRatio_local = self.TPSLRatio

        # Логика закрытия позиций по уровням RSI.
        # Используем атрибуты класса rsi_sell_level и rsi_buy_level.
        if len(self.trades) > 0:
            # Убедимся, что RSI доступен и не NaN для текущей свечи.
            if len(self.data.RSI) > 0 and not np.isnan(self.data.RSI[-1]):
                if self.trades[-1].is_long and self.data.RSI[-1] >= self.rsi_sell_level:
                    self.trades[-1].close()
                elif self.trades[-1].is_short and self.data.RSI[-1] <= self.rsi_buy_level:
                    self.trades[-1].close()

        # Логика открытия позиций на основе сигнала TotalSignal.
        # TotalSignal (signal1) уже учитывает все необходимые параметры.
        # Проверяем, нет ли открытых сделок перед открытием новой.
        # Доступ к последнему значению сигнала осуществляется через индексацию [-1].
        # Убедимся, что сигнал доступен и не NaN.
        if self.signal1 is not None and len(self.signal1) > 0 and not np.isnan(self.signal1[-1]):
             # Убедимся, что цена закрытия не равна 0 для расчета размера позиции.
            if len(self.data.Close) > 0 and self.data.Close[-1] != 0:
                # Рассчитываем размер позиции как долю от начального капитала, используя position_size_percentage.
                # backtesting.py size=X интерпретируется как X% от начального капитала, если не указано иное.
                # Передаем position_size_percentage / 100.0 как размер.
                # Используем position_size_percentage из атрибута класса (установлен Backtest'ом)
                position_size_fraction = self.position_size_percentage / 100.0

                if self.signal1[-1] == 2 and len(self.trades) == 0: # Сигнал на покупку
                    # Рассчитываем уровни стоп-лосса и тейк-профита.
                    sl1 = self.data.Close[-1] - slatr
                    tp1 = self.data.Close[-1] + slatr * TPSLRatio_local
                    # Открываем позицию на покупку с рассчитанным размером.
                    self.buy(sl=sl1, tp=tp1, size=position_size_fraction)

                elif self.signal1[-1] == 1 and len(self.trades) == 0: # Сигнал на продажу
                    # Рассчитываем уровни стоп-лосса и тейк-профита.
                    sl1 = self.data.Close[-1] + slatr
                    tp1 = self.data.Close[-1] - slatr * TPSLRatio_local
                    # Открываем позицию на продажу с рассчитанным размером.
                    self.sell(sl=sl1, tp=tp1, size=position_size_fraction)


# Обновляем выполнение бэктеста, используя глобальные переменные из ячейки параметров.
# Backtest автоматически связывает параметры стратегии (атрибуты класса MyStrat)
# с глобальными переменными, имеющими те же имена.
bt = Backtest(dfpl, MyStrat,
              cash=initial_cash,       # Используем initial_cash из параметров (для Backtest)
              margin=margin,           # Используем margin из параметров (для Backtest)
              commission=0.00
             )

# Запускаем бэктест
stat = bt.run()

# Вывод статистики
display(stat)

Backtest.run:   0%|          | 0/224988 [00:00<?, ?bar/s]

,0
Start,2019-09-30 00:00:00
End,2022-09-30 20:55:00
Duration,1096 days 20:55:00
Exposure Time [%],10.048047
Equity Final [$],39462.4218
Equity Peak [$],41589.611127
Return [%],294.624218
Buy & Hold Return [%],-10.401623
Return (Ann.) [%],44.139366
Volatility (Ann.) [%],26.581361


In [105]:
#оставить
bt.plot(show_legend=False)

/usr/local/lib/python3.11/dist-packages/backtesting/_plotting.py:141: UserWarning:

Data contains too many candlesticks to plot; downsampling to '4h'. See `Backtest.plot(resample=...)`



GridPlot(id='p2597', ...)